# 03 - Coupled Oscillators

**Overview** 

This notebook guides you through ...  

In [ ]:
# @title Modules Setup { display-mode: "form" }
import numpy as np
# Install Plotly (if not already)
!pip install -q plotly > /dev/null
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
!pip install -q rdkit > /dev/null
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw

**Problem** 

Lorem Ipsum ... 

**Model**

Lorem Ipsum ...

>Smart question?

Lorem Ipsum ...

**Questions**

Before you run any simulation, answer the following question(s):

1. Something
2. Seomthing else

Run the simulation, change the parameters, and run the simulation again as many times as needed to answer the following question(s):

3. Other questions

In [ ]:
# @title Simulation Parameters  { display-mode: "form" }
AB_eq_bond_length = 1.0 # @param {type:"number"}
AC_eq_bond_length = 1.0 # @param {type:"number"}
AB_bond_strenght = 1.0 # @param {type:"number"}
AC_bond_strenght = 1.0 # @param {type:"number"}
A_mass = "unit" # @param ["unit", "infinite"]
B_mass = 1  # @param {type:"number"}
C_mass = 1  # @param {type:"number"}
Temperature = 1.0  # @param {type:"number"}
dt = 0.01  # @param {type:"number"}
nsteps = 10000  # @param {type:"integer"}
frame_stride = 100 # @param {type:"integer"}
total_time = nsteps * dt
remove_com = True # @param {type:"boolean"}

In [ ]:
# @title Run and Visualize the Simulation  { display-mode: "form" }
# --- functions to compute forces and energies ---
def compute_forces(nparticles, positions, springs):
    forces = np.zeros(nparticles)
    for (i, j, props) in springs:
        k = props["k"]
        L0 = props["L0"]
        # displacement
        dx = positions[i] - positions[j]
        dist = abs(dx)
        # avoid division by zero
        if dist == 0:
            continue  
        # Hooke’s law
        F = -k * (dist - L0) * (dx / dist)  
        # apply equal and opposite forces
        forces[i] += F
        forces[j] -= F
    return forces

def kinetic(velocities, masses):
    return 0.5 * np.sum(masses * velocities**2)

def potential(positions, springs):
    U = 0.0
    for (i, j, props) in springs:
        k = props["k"]
        L0 = props["L0"]
        # displacement
        dx = positions[i] - positions[j]
        dist = abs(dx)
        # potential energy
        U += 0.5 * k * (dist - L0)**2
    return U

# Initial conditions
A_x = 0.0
B_x = A_x + AB_eq_bond_length + np.random.randn()*0.1
C_x = A_x - AC_eq_bond_length + np.random.randn()*0.1

nparticles = 3
positions = np.array([A_x, B_x, C_x])
masses = np.array([1e10 if A_mass=="infinite" else 1, B_mass, C_mass])
velocities = np.random.randn(nparticles) * np.sqrt(2*Temperature/masses) * 0.1
if remove_com:
    vcom = np.sum(velocities * masses) / np.sum(masses)
    velocities -= vcom

nspring = 2
springs = [
    (0, 1, {"k": AB_bond_strenght, "L0": AB_eq_bond_length}),
    (0, 2, {"k": AC_bond_strenght, "L0": AC_eq_bond_length}),
]

# Run a dynamic simulation for the animation

# -- Time setup ---
kin = kinetic(velocities, masses)
pot = potential(positions, springs)
time = np.arange(0, total_time, dt)
trajectory = []
energy = []
# -- Loop over time ---
for t in time:
    trajectory.append(positions.copy())
    energy.append((kin, pot, kin + pot))

    forces = compute_forces(nparticles, positions, springs)

    # Update positions and velocities using Velocity Verlet
    positions += velocities * dt + 0.5 * forces / masses * dt**2
    new_forces = compute_forces(nparticles, positions, springs)
    velocities += 0.5 * (forces + new_forces) / masses * dt
    # Remove center of mass motion
    if remove_com:
        vcom = np.sum(velocities * masses) / np.sum(masses)
        velocities -= vcom

    kin = kinetic(velocities, masses)
    pot = potential(positions, springs)

# -- end of time loop --
trajectory = np.array(trajectory)  # shape (nt, nparticles)


In [ ]:
colors = ["red", "blue", "green"]

# --- build figure with 2 rows (time series + animation) ---
fig = make_subplots(
    rows=2, cols=1,
    shared_xaxes=False,
    row_heights=[0.4, 0.6],
    vertical_spacing=0.12,   # more spacing
    subplot_titles=("Particle positions vs time", "")  # no title for bottom plot
)
# --- Bottom subplot: initial positions + springs (animated later) ---
init_pos = trajectory[0]
particles = go.Scatter(
    x=init_pos, y=[0]*nparticles,
    mode="markers",
    marker=dict(size=24, color=colors),  # bigger circles
    name="particles"
)
fig.add_trace(particles, row=2, col=1)

# --- Top subplot: static trajectories (never animated) ---
for i in range(nparticles):
    fig.add_trace(
        go.Scatter(
            x=trajectory[:, i], y=time,
            mode="lines", line=dict(color=colors[i]),
            name=f"Particle {i}"
        ),
        row=1, col=1
    )

# --- Build frames for animation ---
frames = []
for i in range(0, nsteps, frame_stride):
    pos = trajectory[i]
    frames.append(go.Frame(data=[
        # bottom subplot: moving particles
        go.Scatter(
            x=pos, y=[0]*nparticles,
            mode="markers",
            marker=dict(size=16, color=colors),
            showlegend=False
        ),
        # top subplot: partial trajectories up to time i
        go.Scatter(x=trajectory[:i,0], y=time[:i], mode='lines', line=dict(color=colors[0])),
        go.Scatter(x=trajectory[:i,1], y=time[:i], mode='lines', line=dict(color=colors[1])),
        go.Scatter(x=trajectory[:i,2], y=time[:i], mode='lines', line=dict(color=colors[2]))
    ]))

# --- Layout ---
fig.update_layout(
    height=700,
    plot_bgcolor="white",   # remove bluish background
    # Top subplot (row 1): positions on x, time on y
    xaxis=dict(range=[trajectory.min()-1, trajectory.max()+1],
               showticklabels=False, title=None),
    yaxis=dict(title="time", range=[time.min(), time.max()]),
    # Bottom subplot (row 2): clean style
    xaxis2=dict(title="position",
                range=[trajectory.min()-1, trajectory.max()+1],
                showgrid=False, zeroline=False, showline=False),
    yaxis2=dict(visible=False, range=[-1, 1],
                showgrid=False, zeroline=False, showline=False),
    updatemenus=[{
        "buttons": [
            {"args": [None, {"frame": {"duration": 50, "redraw": True},
                             "fromcurrent": True, "transition": {"duration": 0}}],
             "label": "▶ Play", "method": "animate"},
            {"args": [[None], {"frame": {"duration": 0, "redraw": False},
                               "mode": "immediate",
                               "transition": {"duration": 0}}],
             "label": "⏸ Pause", "method": "animate"}
        ],
        "direction": "left", "pad": {"r": 10, "t": 87},
        "showactive": False, "type": "buttons",
        "x": 0.1, "xanchor": "right", "y": 1.2, "yanchor": "top"
    }],
)

# --- Attach frames ---
fig.frames = frames

fig.show()


**Homework Assignment**

Pick one (or more) of the following projects:
1. Modify the code to handle a 1D systen with N particles
2. Modify the code to handle a periodic 1D system with 2 particles
3. Modify the code to handle one of the following 2D systems:
    * Water
    * Formaldehyde 
    * 1,3-Butadiene (\(C_{4}H_{6}\))
    * Benzene (only the carbon atoms)  

For the modified code, answer the following questions:

NOTE: It is not necessary that the modified code produces an animation, but you should be able to visualize the results of the simulation in some way. 